<a href="https://colab.research.google.com/github/fjme95/calculo-optimizacion/blob/main/Semana%203/Gradient_Descent_Linear_Regression.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Dependencias

In [61]:
!pip install -U plotly

In [270]:
import numpy as np
import pandas as pd

from sklearn.datasets import make_regression

import plotly.express as px
import plotly.io as pio

In [271]:
pio.templates.default = 'plotly_white'

# Generación de datos

Vamos a generar un problema aleatorio de regresión para probar la implementación que haremos de gradient descent. Recordemos que la **regresión lineal simple** asume que podemos ajustar la variable de respuesta con una recta, esto es, $y = \beta_0 + \beta_1x + \epsilon$, donde $\beta_0$ y $\beta_1$ son los **coeficientes** de la regresión que vamos a ajustar a los datos, al primero también se le conoce como **sesgo**, **intercepto** o **bias**; y $\epsilon$ es el **error** del ajuste.

In [272]:
np.random.seed(19)

bias = np.random.randint(0, 100)
X, y, coef = make_regression(n_features = 1, random_state = 10, noise = 20, bias = bias, coef=True)
print(X[:, 0].shape, y.shape, bias, coef)

(100,) (100,) 93 29.972987242456284


Visualizaremos los datos que se generaron

In [273]:
px.scatter(x = X[:, 0], y = y)

# Primera implementación de Gradient Descent

Nuestro objetivo es encontrar los coeficientes que mejor se ajusten a los datos minimizando el error. Para este caso, vamos a ocupar el **Error Cuadrático Medio** (MSE), Para la matriz de características de $n \times 1$

\begin{align}
    \mathbf{X} = \left[
        \begin{matrix}
            x_1 \\ \vdots \\ x_n
        \end{matrix}
    \right],
\end{align}

el vector de respuesta

\begin{align}
    y = \left[
        \begin{matrix}
            y_1 \\ \vdots \\ y_n
        \end{matrix}
    \right],
\end{align}

y el vector de coeficientes, 

\begin{align}
    \beta = \left[
        \begin{matrix}
            \beta_0 \\ \beta_1
        \end{matrix}
    \right],
\end{align}

el MSE para la regresión lineal simple es 

$$
MSE = \frac{1}{n}\sum_{i=1}^n\left( \beta_0+\beta_1x_i-y_i \right)^2
$$

Lo vamos a ver como una función de pérdida, esto es, la vamos a llamar $J(\beta)$

$$
J(\mathbf{\beta}) = \frac{1}{n}\sum_{i=1}^n\left( \beta_0+\beta_1x_i-y_i \right)^2.
$$

Para ocupar Gradient Descent, necesitamos el gradiente de la función de pérdida que en este caso es

\begin{align}
    \nabla J(\mathbf{\beta}) = \left[
        \begin{matrix}
            \frac{2}{n}\sum_{i=1}^n\left( \beta_0+\beta_1x_i-y_i \right) \\
            \frac{2}{n}\sum_{i=1}^n\left( \beta_0+\beta_1x_i-y_i \right) x_i
        \end{matrix}
    \right],
\end{align}

Los coeficientes de van a actualizar de la siguiente manera,

\begin{align}
    \beta^{(t)} = \beta^{(t-1)} - \gamma \nabla J\left( \beta^{(t-1)} \right),
\end{align}

donde $\gamma$ es la **tasa de aprendizaje (learning rate)**. De manera explícita, $$\beta_0^{(t)} = \beta_0^{(t-1)} - \gamma \frac{2}{n}\sum_{i=1}^n\left( \beta_0^{(t-1)}+\beta_1^{(t-1)}x_i-y_i \right)$$ y $$\beta_1^{(t)} = \beta_1^{(t-1)} - \gamma \frac{2}{n}\sum_{i=1}^n\left( \beta_0^{(t-1)}+\beta_1^{(t-1)}x_i-y_i \right)x_i$$

Notemos que se agregó $(t)$ a la notación, estos lo podemos leer como "*El valor de la variable en la t-ésima iteración*",


Creamos una función para calcular el error cuadrático medio entre dos numpy.array cualquiera.

## Función de pérdida

In [274]:
def mse(y, y_hat):
    return 1/(2*len(y))*np.sum(np.power(y_hat - y, 2))

## Gradient Step

Creamos una función que va a hacer la actualización de las betas. 

Algoritmo:

1. Definimos ```n``` igual a la cantidad de observaciones que tenemos.
2. Creamos dos variables (```beta0_sum``` y ```beta1_sum```) que van a almacenar el valor de las sumas en el gradiente
3. Para cada observación

    1. Agregamos el valor de la suma de la primera función en el gradiente a ```beta0_sum```
    2. Agregamos el valor de la suma del segunda función en el gradiente a ```beta1_sum```

4. Actualizamos las betas con gradient descent.

In [275]:
def gradient_step(x, y, beta, learning_rate):
    n = len(y)

    beta0_sum = 0
    beta1_sum = 0

    for i in range(n):
        beta0_sum += beta[0] + beta[1] * x[i] - y[i]
        beta1_sum += (beta[0] + beta[1] * x[i] - y[i]) * x[i]

    new_beta = np.zeros(2)
    new_beta[0] = beta[0] - learning_rate * (2/n) * beta0_sum
    new_beta[1] = beta[1] - learning_rate * (2/n) * beta1_sum

    return new_beta

## Gradient Descent

Ahora creamos la función que se va a encargar de iterar para encontrar $\beta$ que minimice nuestro error usando la función creada en la celda anterior.

In [276]:
def gradient_descent(x, y, learning_rate = 1e-4, maxiter = 50, verbose = True):
    beta = np.zeros(2)


    for i in range(maxiter):
        loss = mse(y, beta[0] + beta[1]*x)
        if verbose:
            print(f'i: {i}\tloss: {np.round(loss, 3)}\tbeta:{beta}')
        beta = gradient_step(x, y, beta, learning_rate)

    return beta

## Ajuste y evaluación

Ahora, sólo falta ejecutar las función creadas para obtener las betas que mejor se ajusten a nuestros datos.

In [277]:
beta = gradient_descent(X[:, 0], y, learning_rate=0.1)
beta

i: 0	loss: 5292.123	beta:[0. 0.]
i: 1	loss: 3377.927	beta:[19.3090787   7.39600286]
i: 2	loss: 2180.367	beta:[34.63886849 13.09268676]
i: 3	loss: 1430.832	beta:[46.812218   17.47322842]
i: 4	loss: 961.497	beta:[56.48132    20.83558494]
i: 5	loss: 667.47	beta:[64.16319618 23.41125076]
i: 6	loss: 483.174	beta:[70.26778697 25.37991915]
i: 7	loss: 367.591	beta:[75.12019058 26.88093693]
i: 8	loss: 295.06	beta:[78.97827231 28.02224901]
i: 9	loss: 249.514	beta:[82.04660985 28.88737914]
i: 10	loss: 220.895	beta:[84.48753873 29.54087463]
i: 11	loss: 202.899	beta:[86.42990215 30.03255044]
i: 12	loss: 191.573	beta:[87.97598344 30.40079521]
i: 13	loss: 184.44	beta:[89.20699951 30.67514461]
i: 14	loss: 179.943	beta:[90.18745479 30.87828289]
i: 15	loss: 177.105	beta:[90.9685925  31.02759846]
i: 16	loss: 175.313	beta:[91.59113104 31.13639166]
i: 17	loss: 174.18	beta:[92.08743387 31.21481172]
i: 18	loss: 173.463	beta:[92.48323056 31.27058277]
i: 19	loss: 173.008	beta:[92.79898209 31.30956587]
i: 20	lo

array([94.05459535, 31.34589136])

Como generamos los datos, conocemos los valores reales de las betas. Podemos ver qué las diferencias entre los valores ajustados y los valores reales.

In [278]:
print(
    '                        sesgo\tcoeficiente\n'
    f'Coeficientes reales:    {np.round(bias, 2)}\t{np.round(coef, 2)}\n', 
    f'Coeficiente obtenidos: {np.round(beta[0], 2)}\t{np.round(beta[1], 2)}'
)

                        sesgo	coeficiente
Coeficientes reales:    93	29.97
 Coeficiente obtenidos: 94.05	31.35


Graficamos los datos y la recta que encontramos usando gradient descent.

In [279]:
import plotly.graph_objs as go
fig = go.Figure()
# Grafica la recta
fig.add_trace(go.Scatter(x = X[:, 0], y = beta[0] + beta[1] * X[:, 0], name=r"$y=\beta_0 + \beta_1 x$"))
# Grafica los puntos
fig.add_trace(go.Scatter(x = X[:, 0], y = y, mode = 'markers', name = 'datos'))

Como podemos observar, se logró ajustar el modelo. 


# Segunda implementación - Un poco de optimización

Observamos que el modelo empezó a converger desde la iteración 22, incluso un poco antes, 

```
i: 20	loss: 172.72	beta:[93.05096412 31.33619348]
i: 21	loss: 172.536	beta:[93.25212682 31.35380495]
i: 22	loss: 172.42	beta:[93.41277725 31.36490514]
i: 23	loss: 172.346	beta:[93.54112128 31.37136356]
```
Podemos optimizar el algoritmo para que se detenga una vez que el cambio en la pérdida es menor a cierto umbral. Actualicemos la función ```gradient_descent``` para contemplar este cambio.

Agregaremos el parámetro ```loss_tol``` para detener el gradient descent si $| J(\beta^{(t)}) - J(\beta^{(t)}) | < loss\_tol$.

In [280]:
def gradient_descent(x, y, learning_rate = 1e-4, maxiter = 50, loss_tol = 1e-3, verbose = True):
    beta = np.zeros(2)

    prev_loss = 0
    for i in range(maxiter):
        loss = mse(y, beta[0] + beta[1]*x)
        if verbose:
            print(f'i: {i}\tloss: {np.round(loss, 3)}\tbeta:{beta}')
        beta = gradient_step(x, y, beta, learning_rate)
        if np.abs(loss - prev_loss) < loss_tol:
            break
        prev_loss = loss


    return beta

beta = gradient_descent(X[:, 0], y, learning_rate=0.1)
beta

i: 0	loss: 5292.123	beta:[0. 0.]
i: 1	loss: 3377.927	beta:[19.3090787   7.39600286]
i: 2	loss: 2180.367	beta:[34.63886849 13.09268676]
i: 3	loss: 1430.832	beta:[46.812218   17.47322842]
i: 4	loss: 961.497	beta:[56.48132    20.83558494]
i: 5	loss: 667.47	beta:[64.16319618 23.41125076]
i: 6	loss: 483.174	beta:[70.26778697 25.37991915]
i: 7	loss: 367.591	beta:[75.12019058 26.88093693]
i: 8	loss: 295.06	beta:[78.97827231 28.02224901]
i: 9	loss: 249.514	beta:[82.04660985 28.88737914]
i: 10	loss: 220.895	beta:[84.48753873 29.54087463]
i: 11	loss: 202.899	beta:[86.42990215 30.03255044]
i: 12	loss: 191.573	beta:[87.97598344 30.40079521]
i: 13	loss: 184.44	beta:[89.20699951 30.67514461]
i: 14	loss: 179.943	beta:[90.18745479 30.87828289]
i: 15	loss: 177.105	beta:[90.9685925  31.02759846]
i: 16	loss: 175.313	beta:[91.59113104 31.13639166]
i: 17	loss: 174.18	beta:[92.08743387 31.21481172]
i: 18	loss: 173.463	beta:[92.48323056 31.27058277]
i: 19	loss: 173.008	beta:[92.79898209 31.30956587]
i: 20	lo

array([94.01030666, 31.3581165 ])

In [281]:
import plotly.graph_objs as go
fig = go.Figure()
# Grafica la recta
fig.add_trace(go.Scatter(x = X[:, 0], y = beta[0] + beta[1] * X[:, 0], name=r"$y=\beta_0 + \beta_1 x$"))
# Grafica los puntos
fig.add_trace(go.Scatter(x = X[:, 0], y = y, mode = 'markers', name = 'datos'))

Observamos que se detuvo en la iteración 33 en lugar de llegar hasta la 50 como la vez pasada y el ajuste sigue siendo bueno a simple vista.

# Tercera implementación - Visualización de la optimización

Esta implementación hará cambios míminos a la implementación, para que podamos ver cómo es que van cambiando las betas y la recta de la regresión.

In [282]:
def gradient_descent(x, y, learning_rate = 1e-4, maxiter = 50, loss_tol = 1e-3, verbose = True):
    beta = np.zeros(2)

    # Se crea un diccionario que va a guardar el historial de los valores
    history = {
        'beta': [beta], 
        'loss': [mse(y, beta[0] + beta[1]*x)], 
        'loss_diff': [np.nan]
    }

    prev_loss = 0
    for i in range(maxiter):
        beta = gradient_step(x, y, beta, learning_rate)
        loss = mse(y, beta[0] + beta[1]*x)

        loss_diff = np.abs(loss - prev_loss)

        history['beta'].append(beta)
        history['loss'].append(loss)
        history['loss_diff'].append(loss_diff)
        
        if verbose:
            print(f'i: {i}\tloss: {np.round(loss, 3)}\tbeta:{beta}')

        if  loss_diff < loss_tol:
            break
        prev_loss = loss


    return beta, history

beta, history = gradient_descent(X[:, 0], y, learning_rate=0.1, verbose = False)
beta

array([93.99916277, 31.36005941])

In [291]:
from plotly.subplots import make_subplots
# fig = make_subplots(rows = 3, cols = 1)
fig = make_subplots(rows = 2, cols = 1)

fig.append_trace(go.Scatter(x = list(range(len(history['beta']))), y = [beta for beta, _ in history['beta']], name = 'bias', mode = 'markers+lines'), row = 1, col = 1)
fig.append_trace(go.Scatter(x = list(range(len(history['beta']))), y = [beta for _, beta in history['beta']], name = 'coef', mode = 'markers+lines'), row = 1, col = 1)

fig.append_trace(go.Scatter(x = list(range(len(history['loss']))), y = history['loss'], name = 'loss', mode = 'markers+lines'), row = 2, col = 1)
# fig.append_trace(go.Scatter(x = list(range(len(history['loss']))), y = history['loss_diff'], name = 'loss'), row = 3, col = 1)


fig.update_layout(
    title = r'Cambios en las betas y la pérdida conforme se iteró con gradient descent', 
    xaxis_title = 'Iteración', 
    yaxis_title = 'Valor'
    )
fig.show()

In [299]:
plot_data = pd.DataFrame(columns = ['x', 'y_hat', 'it'])

for i, beta in enumerate(history['beta']):
    plot_data = plot_data.append(pd.DataFrame([X[:, 0], beta[0] + beta[1] * X[:, 0], np.repeat(i, len(X))], index= ['x', 'y_hat', 'it']).T)

fig = px.line(plot_data, x = 'x', y = 'y_hat', animation_frame='it')
fig.add_scatter(x = X[:, 0], y = y, mode = 'markers', name = 'datos')

# Extra

Podemos hacer cualquier hipótesis sobre cómo se comportan los datos, en el caso de la regresión lineal, la hipótesis es $\hat{y}^{(i)} = h_\beta(x^{(i)}) = \beta^Tx^{(i)}$. En este ejemplo, la hipótesis es $\hat{y}^{(i)} = h_\beta(x^{(i)}) = \beta^T(x^{(i)})^2$. Se hacen los cambios necesarios al gradiente y se ajusta el modelo.

**Nota**: En la *vida real* se haría algo que se conoce como *feature engineering* para tratar de ocupar métodos usuales. Lo visto en esta sección es sólo es para fines demostrativos

In [301]:
#np.random.seed(123)
n = 100
beta_0 = np.random.random() * 40
beta_1 = np.random.random() * 40
err = np.random.normal(scale = 20, size = n)

X = np.random.rand(n)*10 - 5
y = beta_0 + beta_1 * X**2 + err

px.scatter(x = X, y = y)

In [302]:
def gradient_step(x, y, beta, learning_rate):
    n = len(y)

    beta0_sum = 0
    beta1_sum = 0

    # Aqui fue donde se hicieron los cambios. Se elevo x al cuadrado
    for i in range(n):
        beta0_sum += beta[0] + beta[1] * x[i]**2 - y[i]
        beta1_sum += (beta[0] + beta[1] * x[i]**2 - y[i]) * x[i]**2

    new_beta = np.zeros(2)
    new_beta[0] = beta[0] - learning_rate * (2/n) * beta0_sum
    new_beta[1] = beta[1] - learning_rate * (2/n) * beta1_sum

    return new_beta

In [303]:
def gradient_descent(x, y, learning_rate = 1e-4, maxiter = 20, loss_tol = 1e-4, verbose = True):
    beta = np.zeros(2)

    prev_loss = 0
    for i in range(maxiter):
        # Aqui también se elevó x al cuadrado
        loss = mse(y, beta[0] + beta[1]*x**2)
        if verbose:
            print(f'i: {i}\tloss: {np.round(loss, 3)}\tbeta:{beta}')
        beta = gradient_step(x, y, beta, learning_rate)
        
        if np.abs(loss - prev_loss) < loss_tol:
            break
        prev_loss = loss


    return beta

In [304]:
beta = gradient_descent(X, y, learning_rate=0.001, maxiter=10000, verbose = False)
beta

array([13.25303414, 16.69660782])

In [305]:
plot_data = pd.DataFrame([X, y], index = ['x', 'y']).T.sort_values('x')
plot_data

,x,y
47,-4.958321,397.592202
93,-4.918287,425.865442
54,-4.821561,413.858199
11,-4.490104,319.807201
22,-4.476894,374.647549
...,...,...
46,4.427158,340.910306
55,4.436496,334.585884
24,4.740101,400.424469
39,4.850521,376.240250


In [306]:
import plotly.graph_objs as go
fig = go.Figure()
# Grafica la recta
fig.add_trace(go.Scatter(x = plot_data.x, y = beta[0] + beta[1] * plot_data.x**2, name=r"$y=\beta_0 + \beta_1 x^2$",))
# Grafica los puntos
fig.add_trace(go.Scatter(x = X, y = y, mode = 'markers', name = 'datos'))

# Ejercicios

1. Utilizar esta implementación en los datos del precio de las casas vistos en la semana 2.
2. Graficar la función de pérdida
3. Adaptar la implementación para el caso en el que se tengan 2 carácterísticas
4. Adaptar la implementación para el caso en el que se tengan 3 carácterísticas

## Opcional 

- Graficar el "camino" que se siguió con gradient descent en la pérdida
- Generalizar la implementación para poder hacer regresiones lineales múltiples con cualquier cantidad de características.
- Hacer una animación como la vista en el notebook para ver cómo fue ajustando poco a poco la curva en "Extra"